In [1]:
# Importação das bibliotecas necessárias
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np

In [2]:
# Configuração do webdriver e inicialização do navegador
chrome_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=chrome_service)

# URL da página
url = 'https://www.imovelweb.com.br/apartamentos-aluguel-distrito-federal.html'

# Carregamento da página
driver.get(url)

# Esperar até que os elementos principais estejam carregados
time.sleep(10)

# Extrair o conteúdo HTML da página
page_content = driver.page_source
soup = BeautifulSoup(page_content, 'html.parser')       

# Fechando o navegador
driver.quit()
soup

<html lang="pt-BR"><head>
<link href="https://imgbr.imovelwebcdn.com" rel="preconnect"/>
<link href="https://imgbr.imovelwebcdn.com" rel="dns-prefetch"/>
<link href="https://img10.naventcdn.com" rel="dns-prefetch"/>
<link href="https://img10.naventcdn.com" rel="preconnect"/>
<link as="image" fetchpriority="high" href="https://imgbr.imovelwebcdn.com/avisos/2/29/97/38/68/87/360x266/4409881738.jpg?isFirstImage=true" rel="preload"/><link as="image" fetchpriority="high" href="https://imgbr.imovelwebcdn.com/avisos/2/29/97/38/57/26/360x266/4409793674.jpg?isFirstImage=true" rel="preload"/>
<script async="" src="https://sb.scorecardresearch.com/c2/14366299/cs.js" type="text/javascript"></script><script async="" src="https://analytics.staticiv.com/xG1igYKHF/iva.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-RY7RHKS235&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="//widgets.getsitecontrol.com/97860/script.j

In [5]:
# Encontrar todos os anúncios de imóveis visíveis
anuncios = soup.find_all('div', {'class': 'CardContainer-sc-1tt2vbg-5 fvuHxG'})
print(len(anuncios))

#Visualização da estrutura HTML do primeiro anúncio
anuncios[0]

20


<div class="CardContainer-sc-1tt2vbg-5 fvuHxG"><div class="PostingCardLayout-sc-i1odl-0 egwEUc" data-id="2997386887" data-posting-type="PROPERTY" data-qa="posting PROPERTY" data-to-posting="/propriedades/ap16950-qi-20-otimo-apartamento-com-02-quartos-de-2997386887.html"><div class="PostingCardContainer-sc-i1odl-1 kfOVKt"><div class="GalleryContainer-sc-n2cjqs-0 ckhZnU" data-qa="POSTING_CARD_GALLERY"><div class="lazyload-wrapper" style="width: 100%; height: 100%; position: absolute;"><div class="GalleryContainer-sc-1yqjv7m-0 hKvakS"><div class="multimediaGallery flickity-enabled is-draggable" tabindex="0"><div class="flickity-viewport" style="height: 259.8px; touch-action: pan-y;"><div class="flickity-slider" style="left: 0px; transform: translateX(0%);"><img alt="7a064f465b7483206d8d274367bf1e0f5d710362 · AP16950 - QI 20, Ótimo apartamento com 02 quartos de 68m². Agende a sua visita!" class="is-selected" fetchpriority="high" height="100%" loading="eager" src="https://imgbr.imovelwebcdn

In [38]:
# Extrair as informações desejadas dos anúncios
imoveis = []
for anuncio in anuncios :
    
    titulo = anuncio.find('a', {'href': 'PostingDescription-sc-i1odl-11 fECErU'})
    
    
    try:
        aluguel_elem = anuncio.find('div', {'class': 'Price-sc-12dh9kl-3 geYYII'})
        aluguel = aluguel_elem.get_text(strip=True) if aluguel_elem else np.nan    
        aluguel = aluguel.replace('R$ ','').replace('.','')
        aluguel = float(aluguel)
    except: 
        aluguel = 0

    try:
        condominio_elem = anuncio.find('div', {'class': 'Expenses-sc-12dh9kl-1 iboaIF'})
        condominio = condominio_elem.get_text(strip=True) if condominio_elem else np.nan    
        condominio = condominio.replace('R$ ','').replace('Condomínio','').replace('Condominio','').replace('.','')
        condominio = re.sub(r'\s.*','',condominio)
        condominio = float(condominio)
    except:
        condominio = 0
    
    bairro_elem = anuncio.find('h2', {'class': 'LocationLocation-sc-ge2uzh-2 fziprF'})
    bairro = bairro_elem.get_text(strip=True) if bairro_elem else np.nan

    area_elem = anuncio.find('h3', {'class':'PostingMainFeaturesBlock-sc-1uhtbxc-0 cHDgeO'})
    area = area_elem.get_text(strip=True) if area_elem else np.nan
        
    imoveis.append({
    'Título': titulo,
    'Bairro': bairro,
    'Aluguel': aluguel,
    'Condomínio': condominio,
    'Área': area
    })

# Exibir os dados extraídos
for imovel in imoveis:
    print(imovel)

{'Título': None, 'Bairro': 'Guara I, Guará', 'Aluguel': 2200.0, 'Condomínio': 400.0, 'Área': '68 m² tot.2 quartos1 banheiro1 vaga'}
{'Título': None, 'Bairro': 'Noroeste, Brasília', 'Aluguel': 8500.0, 'Condomínio': 1130.0, 'Área': '109 m² tot.3 quartos2 ban.1 vaga'}
{'Título': None, 'Bairro': 'Asa Sul, Brasília', 'Aluguel': 4800.0, 'Condomínio': 950.0, 'Área': '151 m² tot.3 quartos2 ban.'}
{'Título': None, 'Bairro': 'Aguas Norte, Águas Claras', 'Aluguel': 10500.0, 'Condomínio': 2200.0, 'Área': '361 m² tot.5 quartos7 ban.3 vaga'}
{'Título': None, 'Bairro': 'Águas Claras, Brasília', 'Aluguel': 2800.0, 'Condomínio': 696.0, 'Área': '80 m² tot.2 quartos3 ban.1 vaga'}
{'Título': None, 'Bairro': 'Asa Norte, Brasília', 'Aluguel': 2800.0, 'Condomínio': 660.0, 'Área': '34 m² tot.1 quartos1 banheiro1 vaga'}
{'Título': None, 'Bairro': 'Setor de Hotéis e Turismo Norte, Brasília', 'Aluguel': 1900.0, 'Condomínio': 0, 'Área': '38 m² tot.1 quartos1 banheiro1 vaga'}
{'Título': None, 'Bairro': 'Asa Norte,